# Functional test with FTDI - ADF4351
- https://www.analog.com/media/en/technical-documentation/data-sheets/ADF4351.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = [' ']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from utilities.adapters import peripherals
from signal_generators.adf435x import ADF4351 
from utilities.shift_register import ShiftRegister

Populating the interactive namespace from numpy and matplotlib


In [5]:
import pandas as pd

#https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

## Debug mode?

In [6]:
cls = ADF4351 

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators

In [8]:
with_hardware_device = True

if with_hardware_device:
    _clk = peripherals.Pin.get_Ftdi_pin(pin_id = 4)
    _data = peripherals.Pin.get_Ftdi_pin(pin_id = 1)
    _ss = peripherals.Pin.get_Ftdi_pin(pin_id = 3) 
    _spi = ShiftRegister(stb_pin = _ss, clk_pin = _clk, data_pin = _data, polarity = 0)
    
else:
    _spi = _ss = None  # using None for testing without actual hardware device.

bus = peripherals.SPI(_spi, _ss)
adf = cls(bus)  

In [8]:
ad_1500M = [0x3C0000, 0x80087D1, 0x30041C2, 0xE404B3, 0x932224, 0X580005] 
my_config_1500M = [0x3C0000, 134252497, 50348482, 14943411, 9642532, 5767173] 

df = adf.map.compare_values_sets_pd(enumerate(ad_1500M), enumerate(my_config_1500M))
df[df.different == 1]

,register,address,default_value,element_name,idx_lowest_bit,n_bits,read_only,value,value_2,different


In [9]:
# adf.map.load_values(enumerate(ad_1500M))
# adf.write_all_registers()

In [10]:
# adf.map.load_values(enumerate(my_config_1500M))
# adf.write_all_registers()

In [11]:
adf.set_frequency(2e9, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [12]:
adf.set_frequency(1.5e9, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [13]:
adf.set_frequency(1.0e9, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [14]:
adf.rf_n_divider.step(3)

In [15]:
adf.set_frequency(500e6, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [16]:
adf.set_frequency(435e6, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [17]:
adf.set_frequency(434375000, channel_resolution=100e3)#, rf_divider_as=2)
# adf.current_dividers

In [18]:
adf.set_frequency(50.2e6, channel_resolution=100e3)
# adf.current_dividers

In [19]:
ad_50M = [0x400000, 0x80087D1, 0x3004FC2, 0x6004B3, 0xEC8224, 0x580005]
my_config_50M = [4194304, 134252497, 50352066, 6292659, 15499812, 5767173]
my_config_50_2M = [4195328, 134252497, 50351682, 1203, 15499812, 5767173]

df = adf.map.compare_values_sets_pd(enumerate(ad_50M), enumerate(my_config_50_2M))
df[df.different == 1]

,register,address,default_value,element_name,idx_lowest_bit,n_bits,read_only,value,value_2,different
2,REGISTER_0,0,0,FRAC,3,12,False,0,128,1
18,REGISTER_2,2,0,LDF,8,1,False,1,0,1
19,REGISTER_2,2,0,LDP,7,1,False,1,0,1
27,REGISTER_3,3,0,ABP,22,1,False,1,0,1
28,REGISTER_3,3,0,Charge_Cancelation,21,1,False,1,0,1


In [20]:
adf.map.load_values(enumerate(ad_1500M))
adf.write_all_registers()

In [21]:
adf.map.load_values(enumerate(my_config_50_2M))
adf.write_all_registers()

## BFSK test

In [22]:
adf.reset()

adf.rf_n_divider._set_channel_resolution(100e3)

reg_0_1500M = 0x3C0000 + 0x40
reg_0_1500_2M = 0x3C0020
reg_values = (reg_0_1500M, reg_0_1500_2M)
seq = np.random.choice((0, 1), size = 100)

for s in seq:
    adf.map._registers[0].load_value(reg_values[s])
    adf._write_register_0()

## Current Configuration

In [23]:
RF_out_min = 2.2e9 / 64
RF_out_max = 4.4e9
RF_out_min, RF_out_max

(34375000.0, 4400000000.0)

In [24]:
# adf.set_frequency(RF_out_min, channel_resolution = 100e3, rf_divider_as = None)
adf.set_frequency(35e6, channel_resolution = 100e3, rf_divider_as = None)
# adf.set_frequency(1.599899e9) 
# adf.set_frequency(RF_out_max, channel_resolution = 100e3, rf_divider_as = None)

# adf.phaser.set_phase(90)

df_dividers, df_controls = adf.current_configuration
df_dividers

,type,source_type,source_freq,my_divider,divider_equivalent,is_integer,my_freq
0,_ReferenceInput,NaN,NaN,NaN,NaN,NaN,25000000
1,_ReferenceDoubler,_ReferenceInput,2.500000e+07,2.000,NaN,True,50000000
2,_R_Counter,_ReferenceDoubler,5.000000e+07,1.000,NaN,True,50000000
3,_ReferenceDivider,_R_Counter,5.000000e+07,2.000,NaN,True,25000000
4,_PhaseFrequencyDetector,_ReferenceDivider,2.500000e+07,1.000,NaN,True,25000000
5,_RF_N_Divider,_VCO,2.239900e+09,89.596,89.596,False,25000000
6,_VCO,_PhaseFrequencyDetector,2.500000e+07,89.596,NaN,False,2239900000
7,_RF_Divider,_VCO,2.239900e+09,64.000,NaN,True,34998438
8,_RF_Output,_RF_Divider,3.499844e+07,1.000,NaN,True,34998438
9,_AuxOutput,_VCO,2.239900e+09,1.000,NaN,True,2239900000


In [25]:
df_controls

,type,source_type,source_freq,phase_adjust_enabled,phase,prescaler,my_divider,is_integer,my_freq,enabled,mode,charge_pump_current,three_state_enabled,cancelation_enabled
0,_Phaser,_RF_N_Divider,25000000,False,1.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,_Prescaler,_VCO,2239900000,NaN,NaN,8/9,0.888889,False,1991022222,NaN,NaN,NaN,NaN,NaN
2,_MuxOut,THREE_STATE,None,NaN,NaN,NaN,1.000000,True,None,NaN,NaN,NaN,NaN,NaN
3,_BandSelectClockDivider,_ReferenceDivider,25000000,NaN,NaN,NaN,50.000000,True,500000,True,HIGH,NaN,NaN,NaN
4,_ClockDivider,_ReferenceDivider,25000000,NaN,NaN,NaN,150.000000,True,166667,NaN,CLOCK_DIVIDER_OFF,NaN,NaN,NaN
5,_NoiseControl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LOW_NOISE_MODE,NaN,NaN,NaN
6,_ChargePump,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.31,False,False


## Find Integer N-dividers for a Frequency

In [26]:
adf.find_integer_N_dividers(freq_desired = 1.500e9,
                            ref_doubled_by_2 = True, ref_divided_by_2 = True, 
                            rf_divider_as = None, 
                            torance_hz = 1, freq_ref = 25e6)

[((2, 1, 2, 120.0, 2),
  (25000000.0,
   50000000,
   50000000,
   25000000,
   3000000000,
   1500000000,
   1500000000)),
 ((2, 2, 2, 240.0, 2),
  (25000000.0,
   50000000,
   25000000,
   12500000,
   3000000000,
   1500000000,
   1500000000)),
 ((2, 4, 2, 480.0, 2),
  (25000000.0,
   50000000,
   12500000,
   6250000,
   3000000000,
   1500000000,
   1500000000)),
 ((2, 5, 2, 600.0, 2),
  (25000000.0,
   50000000,
   10000000,
   5000000,
   3000000000,
   1500000000,
   1500000000)),
 ((2, 8, 2, 960.0, 2),
  (25000000.0,
   50000000,
   6250000,
   3125000,
   3000000000,
   1500000000,
   1500000000)),
 ((2, 10, 2, 1200.0, 2),
  (25000000.0,
   50000000,
   5000000,
   2500000,
   3000000000,
   1500000000,
   1500000000)),
 ((2, 16, 2, 1920.0, 2),
  (25000000.0,
   50000000,
   3125000,
   1562500,
   3000000000,
   1500000000,
   1500000000)),
 ((2, 20, 2, 2400.0, 2),
  (25000000.0,
   50000000,
   2500000,
   1250000,
   3000000000,
   1500000000,
   1500000000)),
 ((2, 25, 2,

In [27]:
adf.find_integer_N_dividers(freq_desired = 50e6,
                            ref_doubled_by_2 = True, ref_divided_by_2 = True, 
                            rf_divider_as = None, 
                            torance_hz = 1, freq_ref = 25e6)

[((2, 1, 2, 128.0, 64),
  (25000000.0, 50000000, 50000000, 25000000, 3200000000, 50000000, 50000000)),
 ((2, 2, 2, 256.0, 64),
  (25000000.0, 50000000, 25000000, 12500000, 3200000000, 50000000, 50000000)),
 ((2, 4, 2, 512.0, 64),
  (25000000.0, 50000000, 12500000, 6250000, 3200000000, 50000000, 50000000)),
 ((2, 5, 2, 640.0, 64),
  (25000000.0, 50000000, 10000000, 5000000, 3200000000, 50000000, 50000000)),
 ((2, 8, 2, 1024.0, 64),
  (25000000.0, 50000000, 6250000, 3125000, 3200000000, 50000000, 50000000)),
 ((2, 10, 2, 1280.0, 64),
  (25000000.0, 50000000, 5000000, 2500000, 3200000000, 50000000, 50000000)),
 ((2, 16, 2, 2048.0, 64),
  (25000000.0, 50000000, 3125000, 1562500, 3200000000, 50000000, 50000000)),
 ((2, 20, 2, 2560.0, 64),
  (25000000.0, 50000000, 2500000, 1250000, 3200000000, 50000000, 50000000)),
 ((2, 25, 2, 3200.0, 64),
  (25000000.0, 50000000, 2000000, 1000000, 3200000000, 50000000, 50000000)),
 ((2, 32, 2, 4096.0, 64),
  (25000000.0, 50000000, 1562500, 781250, 32000000

## Member functions test

In [28]:
# for f in dir(cls):
#     if not f.startswith('_'):
#         print('adf.{}()'.format(f))

In [29]:
adf.init()

In [30]:
adf.start()

In [31]:
adf.reset() 

In [32]:
adf.apply_signal(freq = 1.5002e9)

In [33]:
df_dividers, df_controls = adf.current_configuration
df_dividers

,type,source_type,source_freq,my_divider,divider_equivalent,is_integer,my_freq
0,_ReferenceInput,NaN,NaN,NaN,NaN,NaN,25000000
1,_ReferenceDoubler,_ReferenceInput,2.500000e+07,2.000000,NaN,True,50000000
2,_R_Counter,_ReferenceDoubler,5.000000e+07,1.000000,NaN,True,50000000
3,_ReferenceDivider,_R_Counter,5.000000e+07,2.000000,NaN,True,25000000
4,_PhaseFrequencyDetector,_ReferenceDivider,2.500000e+07,1.000000,NaN,True,25000000
5,_RF_N_Divider,_VCO,3.000397e+09,120.015873,120.015873,False,25000000
6,_VCO,_PhaseFrequencyDetector,2.500000e+07,120.015873,NaN,False,3000396825
7,_RF_Divider,_VCO,3.000397e+09,2.000000,NaN,True,1500198412
8,_RF_Output,_RF_Divider,1.500198e+09,1.000000,NaN,True,1500198412
9,_AuxOutput,_VCO,3.000397e+09,1.000000,NaN,True,3000396825


In [34]:
df_controls

,type,source_type,source_freq,phase_adjust_enabled,phase,prescaler,my_divider,is_integer,my_freq,enabled,mode,charge_pump_current,three_state_enabled,cancelation_enabled
0,_Phaser,_RF_N_Divider,25000000,False,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,_Prescaler,_VCO,3000396825,NaN,NaN,8/9,0.888889,False,2667019400,NaN,NaN,NaN,NaN,NaN
2,_MuxOut,THREE_STATE,None,NaN,NaN,NaN,1.000000,True,None,NaN,NaN,NaN,NaN,NaN
3,_BandSelectClockDivider,_ReferenceDivider,25000000,NaN,NaN,NaN,50.000000,True,500000,True,HIGH,NaN,NaN,NaN
4,_ClockDivider,_ReferenceDivider,25000000,NaN,NaN,NaN,150.000000,True,166667,NaN,CLOCK_DIVIDER_OFF,NaN,NaN,NaN
5,_NoiseControl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LOW_NOISE_MODE,NaN,NaN,NaN
6,_ChargePump,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.31,False,False


In [35]:
adf.current_dividers

{'d_ref_doubler': 2,
 'd_r_counter': 1,
 'd_ref_divider': 2,
 'd_rf_n_divider': 120.01587301587301,
 'd_rf_divider': 2}

In [36]:
adf.current_frequency

1500198412

In [37]:
adf.current_phase

0.0

In [38]:
adf.enable(False)

In [39]:
adf.enable(True)

In [40]:
adf.enable_output(False)

In [41]:
adf.enable_output(True)

In [42]:
# adf.enable_output_channel(0)

In [43]:
adf.enabled

True

In [44]:
adf.freq_pfd

25000000

In [45]:
adf.freq_resolution

6105.006105006105

In [46]:
adf.phase_resolution

0.08791208791208792

In [47]:
adf.is_virtual_device

True

In [48]:
adf.load_registers(enumerate(ad_1500M))
adf.write_all_registers()

In [49]:
adf.power_downed

False

In [50]:
# adf.read_all_registers()

In [51]:
adf.registers_values

[(0, 3932160),
 (1, 134252497),
 (2, 50348482),
 (3, 14943411),
 (4, 9642532),
 (5, 5767173)]

In [52]:
# adf.select_freq_source(0)

In [53]:
# adf.select_phase_source(0)

In [54]:
adf.set_dividers(2, 1, 2, 120, 2)

In [55]:
adf.set_frequency(1.5002e9)

In [56]:
adf.set_phase(0)

In [57]:
adf.update()

In [58]:
adf.status

OrderedDict([('mclk',
              OrderedDict([('type', '_ReferenceInput'),
                           ('my_freq', 25000000)])),
             ('ref_doubler',
              OrderedDict([('type', '_ReferenceDoubler'),
                           ('source_type', '_ReferenceInput'),
                           ('source_freq', 25000000),
                           ('my_divider', 2),
                           ('is_integer', True),
                           ('my_freq', 50000000)])),
             ('r_counter',
              OrderedDict([('type', '_R_Counter'),
                           ('source_type', '_ReferenceDoubler'),
                           ('source_freq', 50000000),
                           ('my_divider', 1),
                           ('is_integer', True),
                           ('my_freq', 50000000)])),
             ('ref_divider',
              OrderedDict([('type', '_ReferenceDivider'),
                           ('source_type', '_R_Counter'),
                           

In [59]:
adf.print()


<< REGISTER_0 >>  :  ('0x3c0208', '0b1111000000001000001000')
[ Reserved_31 ] :  0
[ INT ]         :  120
[ FRAC ]        :  65
[ Index ]       :  0

<< REGISTER_1 >>      :  ('0x8007ff9', '0b1000000000000111111111111001')
[ Reserved_29 ]     :  0
[ Phase_Adjust ]    :  0
[ Prescaler_Value ] :  1
[ Phase_Value ]     :  0
[ MOD ]             :  4095
[ Index ]           :  1

<< REGISTER_2 >>                   :  ('0x3004042', '0b11000000000100000001000010')
[ Reserved_31 ]                  :  0
[ Low_Noise_and_Low_Spur_Modes ] :  0
[ MUXOUT ]                       :  0
[ Reference_Doubler ]            :  1
[ RDIV2 ]                        :  1
[ R_Counter ]                    :  1
[ Double_Buffer ]                :  0
[ Charge_Pump_Current_Setting ]  :  0
[ LDF ]                          :  0
[ LDP ]                          :  0
[ Phase_Detector_Polarity ]      :  1
[ Power_Down ]                   :  0
[ Charge_Pump_Three_State ]      :  0
[ Counter_Reset ]                :  0
[ Inde

In [60]:
adf.pause()

In [61]:
adf.resume()

In [62]:
adf.stop()

In [63]:
adf.close()